<a href="https://colab.research.google.com/github/threegenie/vgames_project/blob/main/game_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 
from sklearn.preprocessing import StandardScaler 

# read data as df
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/vgames2.csv')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...,...
16593,16594,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,16595,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,16596,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,16597,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [5]:
df = df.drop('Unnamed: 0',axis=1)
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


플랫폼/장르/게임사/년도 로 그룹지어서 각 나라별로 세일즈 비교

In [19]:
df.dtypes

Name            object
Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales        object
EU_Sales        object
JP_Sales        object
Other_Sales     object
dtype: object

In [21]:
pd.pivot_table(df, index = ['Platform'], values = ['NA_Sales','EU_Sales',	'JP_Sales',	'Other_Sales'],aggfunc='sum')

,EU_Sales,JP_Sales,NA_Sales,Other_Sales
Platform,,,,
2600,0.010.020.030.020.0100.040.050.020.120.010.060...,0000000000000000000000K00000000000000000000000...,0.260.290.550.320.210.070.650.90.32.060.181.05...,000.010000.010.0100.0200.010000.010000.010.010...
3DO,000,0.060.020.02,000,000
3DS,0000.020.0500.030.290.080.0400000.040000.310.6...,0.010.080.170.530.01001.960.0600.050.26000.040...,0000.160.140.180.160.460.090.05000.060.120.100...,0000.010.020.010.010.070.020.010000.010.010000...
DC,000.6100.230000.510M0000.050000000000000000.24...,0.370.070.460.060.470.290.050.110.120.050.410....,001.2600.410001.100001.020000000000000000.5200...,000.0800.030000.0800000.020000000000000000.040...
DS,0000000.03M0.050000.060.01000000.020.010.02000...,000.1400.0200.040.250.0500.030.41000.02000.040...,0.040.0400.0200.050.30.3300.0500.520.030.10.21...,0000000.030.040000.0500.010.02000000.01000.060...
GB,0002.260.4800.31001.56001.12000.310.520.2700.6...,0.070.620.334.220.40.140.560.320.541.290.220.0...,00023.21.3900.85002.55003.02000.680.870.6600.9...,0000.580.0800.04000.99000.16000.040.060.030.01...
GBA,0.05000.030.3500.01M0.010.06000.050.130.010.22...,00.080000.080000000000.08000.6200000000000000....,0.1400.010.080.9400.020.040.170.010.010.130.35...,000M00.0200000000.0100.030000.02000000.010.010...
GC,0.010.050.010.010.010.0200.020.010.010.0300.05...,00000000000000000.0100.010000.1500.5600.200000...,0.020.190.030.040.030.060.010.090.050.050.110....,00.0100000000000.01000.020.0300.0400.020.02000...
GEN,00000.250.470.43001.200.550.30.3700.140000.630...,0.140.130.070.1900.20.030.070.050.160.070.110....,00001.751.021.24004.4701.8610.9700.950001.951....,00000.050.070.120M00.1900.080.040.0500.030000....
